In [1]:
import numpy as np
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score


# 1. Завантажити та підготувати датасет

In [2]:
iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['target'] = iris.target

In [3]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [4]:
iris_df.target.value_counts()

0    50
1    50
2    50
Name: target, dtype: int64

# 2. Класифікувати Іриси використовуючи бібліотечні засоби : метод найблищих сусідів

In [5]:
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [6]:
k = 3
knn = KNeighborsClassifier(n_neighbors=k)

knn.fit(X_train, y_train)

KNeighborsClassifier(n_neighbors=3)

In [7]:
y_pred = knn.predict(X_test)

In [8]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точність: {accuracy}')

Точність: 0.98


# 3. Реалізувати метод найблищих сусідів без бібліотечних засобів

In [9]:
class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X_train, y_train):
        self.X_train = X_train
        self.y_train = y_train

    def dist(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def predict(self, X_test):
        y_pred = [self._predict(x) for x in X_test]
        return np.array(y_pred)

    def _predict(self, x):
        distances = [self.dist(x, x_train) for x_train in self.X_train]
        k_indices = np.argsort(distances)[:self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]
        most_common = np.bincount(k_nearest_labels).argmax()
        return most_common

In [10]:
clf_knn = KNN()
clf_knn.fit(X_train, y_train)
y_pred = clf_knn.predict(X_test)

In [11]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точність: {accuracy}')

Точність: 0.98


# 4. Реалізувати задачу використовуючу метод опорних векторів без бібліотечних засобів

In [13]:
def add_bias_feature(a):
    return np.hstack([a, np.ones((a.shape[0], 1))])

class CustomSVM(object):
    
    def __init__(self, learning_rate=0.01, regularization_strength=0.1, epochs=200):
        self.epochs = epochs
        self.learning_rate = learning_rate
        self.regularization_strength = regularization_strength
        self.weights = None

    def fit(self, X_train, y_train):
        X_train = add_bias_feature(X_train)
        num_samples, num_features = X_train.shape
        self.weights = np.random.normal(loc=0, scale=0.05, size=num_features)

        for _ in range(self.epochs):
            for i in range(num_samples):
                margin = y_train[i] * np.dot(X_train[i], self.weights)
                if margin >= 1:
                    self.weights -= self.learning_rate * self.regularization_strength * self.weights
                else:
                    self.weights += self.learning_rate * (y_train[i] * X_train[i] - self.regularization_strength * self.weights)

    def predict(self, X):
        X_extended = add_bias_feature(X)
        predictions = np.sign(np.dot(X_extended, self.weights))
        return predictions


In [16]:
X_binary = X[(y == 0) | (y == 1)]
y_binary = y[(y == 0) | (y == 1)]
y_binary[y_binary == 0] = -1  

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X_binary, y_binary, test_size=0.2, random_state=42)

In [18]:
svm = CustomSVM()
svm.fit(X_train, y_train)

In [19]:
y_pred = svm.predict(X_test)

In [20]:
accuracy = accuracy_score(y_test, y_pred)
print(f'Точность: {accuracy}')

Точность: 1.0
